<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=100 align="left">

# AWS Dynamic Credentials
---

## Business Value of Capability
The AWS secrets engine generates AWS access credentials dynamically based on IAM policies. This generally makes working with AWS IAM easier, since it does not involve clicking in the web UI. Additionally, the process is codified and mapped to internal auth methods (such as LDAP). The AWS IAM credentials are time-based and are automatically revoked when the Vault lease expires.


<img src=images/aws-secret-engine.png width=700 >   


### Setup

**Note**:  
* Run > Render All Markdown Cells
* Edit > Clear all output

In [3]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [4]:
vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       dRq6usE3NFp7vjYtgtH8uqQv
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [ ]:
vault status; vault version

### Demo
---
**Enable the Vault AWS secrets engine**  
Example: `vault secrets enable -path=amz -default-lease-ttl=5m -max-lease-ttl=30m aws`

In [6]:
vault secrets enable -path=aws -default-lease-ttl=5m -max-lease-ttl=30m aws

Success! Enabled the aws secrets engine at: aws/


### Configure the credentials that Vault uses to communicate with AWS to generate temporary IAM credentials

In [2]:
source ~/.zshrc; echo $AWS_ACCESS_KEY_ID $AWS_REGION; unset AWS_SESSION_TOKEN; aws sts get-caller-identity

MO4V
{h=
    "UserId": "GC2",
    "Account": "6182",
    "Arn": "arn:aws:iam::82:user/tbagio"
}


In [7]:
vault write aws/config/root \
  access_key=$AWS_ACCESS_KEY_ID \
  secret_key=$AWS_SECRET_ACCESS_KEY \
  region=$AWS_DEFAULT_REGION

Success! Data written to: aws/config/root


In [8]:
vault read aws/config/root

Key             Value
---             -----
access_key      4V
iam_endpoint    n/a
max_retries     -1
region          ap-southeast-1
sts_endpoint    n/a


---
## Use Case: Generate IAM USER
Configure a Vault role that maps to a set of permissions in AWS as well as an AWS credential type.   
When users generate credentials, they are generated against this role.   
**Note:** credential_type=iam_user

In [9]:
vault write aws/roles/infra-ops \
    credential_type=iam_user \
    policy_document=-<<EOF
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": "ec2:*",
      "Resource": "*"
    }
  ]
}
EOF

Success! Data written to: aws/roles/infra-ops



**Reading the role creates the credentials in AWS as an IAM user with those permissions above**

In [10]:
#-output-curl-string always available
vault read aws/creds/infra-ops

Key                Value
---                -----
lease_id           aws/creds/infra-ops/UtBtNFRQyXYWs8pTjD7g4ai0
lease_duration     5m
lease_renewable    true
access_key         VI
secret_key         jp
security_token     <nil>


In [ ]:
#vault unwrap s.TzPCHYip1lGxIRp1KpQ3j3mK

&nbsp;

In [13]:
AWS_ACCESS_KEY_ID= AWS_SECRET_ACCESS_KEY=;
AWS_DEFAULT_REGION=us-east-2; \ 

aws sts get-caller-identity

zsh: command not found:  
{h=
    "UserId": "YLF3US",
    "Account": "6182",
    "Arn": "arn:aws:iam::6182:user/vault-token-infra-ops-1616303938-2276"
}


&nbsp;

In [ ]:
AWS_ACCESS_KEY_ID=  AWS_SECRET_ACCESS_KEY=;
AWS_DEFAULT_REGION=us-east-2; \
aws rds describe-db-instances --query "DBInstances[*].[DBInstanceIdentifier, DBName, DBInstanceStatus, AvailabilityZone, DBInstanceClass]"
#aws ec2 describe-instances --query "Reservations[*].Instances[*].[VpcId, InstanceId, ImageId, InstanceType]"

&nbsp;  
**Check details on a lease, e.g. the remaining lifetime of the credential**
&nbsp;

In [ ]:
vault write sys/leases/lookup lease_id=aws/creds/infra-ops/FCHNpoRALcRfJQmF5qbjeHWK

**Renew lease**

In [ ]:
vault write sys/leases/renew lease_id=aws/creds/infra-ops/7eqGE2tHNbieGqK6M3pjqBHA

**Revoking the lease removes the user**

In [ ]:
vault lease revoke aws/creds/infra-ops/7eqGE2tHNbieGqK6M3pjqBHA

**Revoke all leases**

In [3]:
vault lease revoke -prefix=true aws/creds/infra-ops

All revocation operations queued successfully!


---
## Use Case: ASSUMMED ROLE
Vault also supports ```assumed_role``` and ```federation_token``` credential types.  
**Note:** Due to limitations in AWS, in order to use the federation_token credential type, Vault must be configured with IAM user credentials. AWS does not allow temporary credentials (such as those from an IAM instance profile) to be used.

In [ ]:
vault write aws/roles/deploy \
    role_arns=arn:aws:iam::726182:role/example-role \
    credential_type=assumed_role

&nbsp;

In [ ]:
vault write aws/sts/deploy ttl=60m

&nbsp;

In [ ]:
export AWS_ACCESS_KEY_ID=; 
export  AWS_SECRET_ACCESS_KEY=;
export AWS_SESSION_TOKEN=;
aws sts get-caller-identity
#aws ec2 describe-regions --filters "Name=endpoint,Values=*us*"

### Test the asssumed role capabilities

In [ ]:
AWS_DEFAULT_REGION=us-east-1; aws ec2 describe-instances --query "Reservations[*].Instances[*].[VpcId, InstanceId, ImageId, InstanceType]"

&nbsp;

In [ ]:
AWS_DEFAULT_REGION=us-east-2; aws rds describe-db-instances --query "DBInstances[*].[DBInstanceIdentifier, DBName, DBInstanceStatus, AvailabilityZone, DBInstanceClass]"

### Disable the AWS Secret engine

In [ ]:
vault secrets disable aws/ 

#### TO DO:
- list policies in an assumed role
aws iam list-roles --query "Roles[?RoleName == 'example-role'].[RoleName, Arn]"
aws iam list-role-policies --role-name example-role

**END**